In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
# from keras.datasets import fashion_mnist
from torchviz import make_dot

In [2]:
X = torch.tensor([[1.,2.],[3.,4.],[5.,6.],[7.,8.]]).to('cuda:0')
Y = [[3],[7],[11],[15]]

In [3]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.X = torch.tensor(x).float().to('cuda:0')
        self.Y = torch.tensor(y).float().to('cuda:0')

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

    def __len__(self):
        return len(self.X)

In [4]:
dataset = MyDataset(X,Y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

/tmp/ipykernel_27384/1763244608.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(x).float().to('cuda:0')


In [5]:
torch.cuda.is_available()

True

In [6]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 8)
        self.activation = nn.ReLU()
        self.layer2 = nn.Linear(8,1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        return x

In [7]:
model = MyModel().to('cuda:0')

In [8]:
make_dot(model(X), params=dict(model.named_parameters())).render('dense_network', format='png')

'dense_network.png'

In [9]:
model.layer1.weight

Parameter containing:
tensor([[ 0.6786,  0.0897],
        [ 0.2578, -0.2195],
        [ 0.4021,  0.0193],
        [-0.6559,  0.4298],
        [-0.6764, -0.3558],
        [-0.0907,  0.4690],
        [ 0.2783,  0.5290],
        [-0.6109, -0.2657]], device='cuda:0', requires_grad=True)

In [10]:
loss = nn.MSELoss()

In [11]:
opt = torch.optim.SGD(model.parameters(), lr= 0.000001)

In [12]:
model.train()
loss_list = []
for epoch in range(10000):
    for ix, iy in dataloader:
        opt.zero_grad()
        loss_value = loss(model(ix), iy)
    
        loss_value.backward()
        opt.step()
        loss_list.append(loss_value.detach())

In [13]:
loss_list[-1]

tensor(0.0127, device='cuda:0')

In [14]:
model.forward(torch.tensor([1.,2.]).to('cuda:0'))

tensor([3.1903], device='cuda:0', grad_fn=<ViewBackward0>)

NameError: name 'FLOAT_MIN' is not defined